<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li></ul></div>

# Исследование надёжности заёмщиков
# Описание проекта
Заказчик — кредитный отдел банка. Входные данные от банка — статистика о платёжеспособности клиентов.

**Цель** - разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

***Дополнительный уровень:*** 

Совершить оптимизацию данных для экономия памяти.

# Описание данных
* `children` — количество детей в семье
* `days_employed` — общий трудовой стаж в днях
* `dob_years` — возраст клиента в годах
* `education` — уровень образования клиента
* `education_id` — идентификатор уровня образования
* `family_status` — семейное положение
* `family_status_id` — идентификатор семейного положения
* `gender` — пол клиента
* `income_type` — тип занятости
* `debt` — имел ли задолженность по возврату кредитов
* `total_income` — ежемесячный доход
* `purpose` — цель получения кредита

## Обзор данных

In [1]:
# загрузим библиотеки
import pandas as pd
import numpy as np

In [2]:
# посмотрим на первые 5 строк
df = pd.read_csv('/datasets/data.csv')
df.head(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [3]:
# общая информация
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [4]:
# более подробные данные по используемой памяти
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


In [5]:
'''Исследуем использование памяти разными типами данных. 
Начнём со среднего показателя использования памяти по разным типам данных.'''
for dtype in ['float','int','object']:
    selected_dtype = df.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype, mean_usage_mb))


Average memory usage for float columns: 0.11 MB
Average memory usage for int columns: 0.00 MB
Average memory usage for object columns: 1.69 MB


Эти сведения дают нам понять то, что большая часть памяти уходит на 5 столбцов, хранящих объектные значения.

In [6]:
'''Для проверки минимального и максимального значения, 
подходящего для хранения с использованием каждого целочисленного подтипа, 
можно воспользоваться методом numpy.iinfo(). Рассмотрим пример:'''
int_types = ["uint8", "int8", "int16"]
for it in int_types:
    print(np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------



Тут можно обратить внимание на различие между типами uint (беззнаковое целое) и int (целое число со знаком). Оба типа имеют одинаковую ёмкость, но, при хранении в столбцах только положительных значений, беззнаковые типы позволяют эффективнее расходовать память.

In [7]:
# создадим функцию, которая поможет нам сэкономить немного времени на выяснении сколько памяти используется.

def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f} MB".format(usage_mb)

In [8]:
gl_int = df.select_dtypes(include=['int'])
converted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')

print(mem_usage(gl_int))
print(mem_usage(converted_int))

compare_ints = pd.concat([gl_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

0.00 MB
0.00 MB


,before,after


In [9]:
gl_float = df.select_dtypes(include=['float'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')

print(mem_usage(gl_float))
print(mem_usage(converted_float))

compare_floats = pd.concat([gl_float.dtypes,converted_float.dtypes],axis=1)
compare_floats.columns = ['before','after']
compare_floats.apply(pd.Series.value_counts)

0.33 MB
0.16 MB


,before,after
float32,NaN,2.0
float64,2.0,NaN


In [10]:
optimized_gl = df.copy()

optimized_gl[converted_int.columns] = converted_int
optimized_gl[converted_float.columns] = converted_float

print(mem_usage(df))
print(mem_usage(optimized_gl))

11.28 MB
11.12 MB


In [11]:
'''Для того чтобы понять, где именно мы сможем воспользоваться категориальными данными для снижения потребления памяти,
выясним количество уникальных значений в столбцах, хранящих значения объектных типов:'''
gl_obj = df.select_dtypes(include=['object']).copy()
gl_obj.describe()

,education,family_status,gender,income_type,purpose
count,21525,21525,21525,21525,21525
unique,15,5,3,8,38
top,среднее,женат / замужем,F,сотрудник,свадьба
freq,13750,12380,14236,11119,797


In [12]:
dow = gl_obj.family_status
print(dow.head())

dow_cat = dow.astype('category')
print(dow_cat.head())

0     женат / замужем
1     женат / замужем
2     женат / замужем
3     женат / замужем
4    гражданский брак
Name: family_status, dtype: object
0     женат / замужем
1     женат / замужем
2     женат / замужем
3     женат / замужем
4    гражданский брак
Name: family_status, dtype: category
Categories (5, object): ['Не женат / не замужем', 'в разводе', 'вдовец / вдова', 'гражданский брак', 'женат / замужем']


In [13]:
dow_cat.head().cat.codes

0    4
1    4
2    4
3    4
4    3
dtype: int8

Тут можно заметить то, что каждому уникальному значению назначено целочисленное значение, и то, что столбец теперь имеет тип int8.

In [14]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

2.93 MB
0.02 MB


Потребление памяти до и после преобразования столбца family_status к типу category.

In [15]:
'''Создадим цикл, который перебирает все столбцы, хранящие данные типа object, 
выясняет не превышает ли число уникальных значений в столбцах 50% 
и если это так, преобразует их в тип category.'''
converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

Сравним то, что получилось после оптимизации, с тем, что было раньше:

In [16]:
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([gl_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

12.34 MB
0.11 MB


,before,after
object,5.0,NaN
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0
category,NaN,1.0


In [17]:
# проанализируем то, на какой уровень использования памяти мы вышли, если сравнить то, что получилось
optimized_gl[converted_obj.columns] = converted_obj

print(mem_usage(optimized_gl))

1.10 MB


In [18]:
dtypes = optimized_gl.dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

'''мы возьмём пары ключ/значение из словаря
и аккуратно их выведем
'''
preview = first2pairs = {key:value for key,value in list(column_types.items())}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

{   'children': 'int64',
    'days_employed': 'float32',
    'debt': 'int64',
    'dob_years': 'int64',
    'education': 'category',
    'education_id': 'int64',
    'family_status': 'category',
    'family_status_id': 'int64',
    'gender': 'category',
    'income_type': 'category',
    'purpose': 'category',
    'total_income': 'float32'}


In [21]:
read_and_optimized = pd.read_csv('/datasets/data.csv',dtype=column_types,infer_datetime_format=True)

print(mem_usage(read_and_optimized))
display(read_and_optimized.head())
u_memory = 1 - (1.10 / 11.3)
print(f'Экономия памяти: {u_memory:.2%}')

1.10 MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.672852,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.640625,покупка жилья
1,1,-4024.803711,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.015625,приобретение автомобиля
2,0,-5623.422852,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.953125,покупка жилья
3,3,-4124.747070,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.562500,дополнительное образование
4,0,340266.062500,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.078125,сыграть свадьбу


Экономия памяти: 90.27%
